In [1]:
import pandas as pd
import sys

In [2]:
# Path containing python module for processing Stack Exchange posts.
sys.path.insert(1, '../src')

import se_post_processing as sepp

# Table of Contents
---

### 1. Introduction
### 2. Data processing
### 3. Modeling
### 4. Conclusion

# 1. Introduction
---

In this notebook, we create our final model for predicting whether or not a question on the Data Science Stack Exchange will be answered within 7 days. We first process the text and engineer features from the contents of the post, then we train and evaluate a random forest model.

# 2. Data processing
---

We begin by preparing the Stack Exchange post data for modeling. We need to process the text in the bodies and titles of the questions to engineer features.

In [3]:
# Read post data and insepct first five rows.
posts = pd.read_xml('../data/ds/posts.xml')
posts.head()

,Id,PostTypeId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastActivityDate,Title,Tags,...,ClosedDate,ContentLicense,AcceptedAnswerId,LastEditorUserId,LastEditDate,ParentId,OwnerDisplayName,CommunityOwnedDate,LastEditorDisplayName,FavoriteCount
0,5,1,2014-05-13T23:58:30.457,9,959.0,<p>I've always been interested in machine lear...,5.0,2014-05-14T00:36:31.077,How can I do simple machine learning without h...,<machine-learning>,...,2014-05-14T14:40:25.950,CC BY-SA 3.0,NaN,NaN,None,NaN,None,None,None,NaN
1,7,1,2014-05-14T00:11:06.457,4,503.0,"<p>As a researcher and instructor, I'm looking...",36.0,2014-05-16T13:45:00.237,What open-source books (or other materials) pr...,<education><open-source>,...,2014-05-14T08:40:54.950,CC BY-SA 3.0,10.0,97.0,2014-05-16T13:45:00.237,NaN,None,None,None,NaN
2,9,2,2014-05-14T00:36:31.077,5,NaN,"<p>Not sure if this fits the scope of this SE,...",51.0,2014-05-14T00:36:31.077,None,None,...,None,CC BY-SA 3.0,NaN,NaN,None,5.0,None,None,None,NaN
3,10,2,2014-05-14T00:53:43.273,13,NaN,"<p>One book that's freely available is ""The El...",22.0,2014-05-14T00:53:43.273,None,None,...,None,CC BY-SA 3.0,NaN,NaN,None,7.0,None,None,None,NaN
4,14,1,2014-05-14T01:25:59.677,26,1925.0,<p>I am sure data science as will be discussed...,66.0,2020-08-16T13:01:33.543,Is Data Science the Same as Data Mining?,<data-mining><definitions>,...,None,CC BY-SA 3.0,29.0,322.0,2014-06-17T16:17:20.473,NaN,None,None,None,NaN


In [4]:
# Read comment data and insepct first five rows.
comments = pd.read_xml('../data/ds/comments.xml')
comments.head()

,Id,PostId,Score,Text,CreationDate,UserId,ContentLicense,UserDisplayName
0,5,5,9,this is a super theoretical AI question. An in...,2014-05-14T00:23:15.437,34.0,CC BY-SA 3.0,None
1,6,7,4,List questions are usually not suited for Stac...,2014-05-14T00:38:19.510,51.0,CC BY-SA 3.0,None
2,9,7,3,This question appears to be off-topic because ...,2014-05-14T01:16:12.623,66.0,CC BY-SA 3.0,None
3,12,15,3,This question is far too broad. It may be salv...,2014-05-14T02:00:22.797,51.0,CC BY-SA 3.0,None
4,13,10,2,"Nice one, @Nicholas... Another book from Hasti...",2014-05-14T02:16:20.503,24.0,CC BY-SA 3.0,None


In [5]:
# Convert `CreationDate` to date time.
posts['CreationDate'] = pd.to_datetime(posts['CreationDate'])
comments['CreationDate'] = pd.to_datetime(comments['CreationDate'])

---

We need to separate the question posts from other kinds of posts and create a column which stores our target variable: whether or not a question was answered within 7 days of posting.

In [6]:
# Isolate questions in a separate data frame.
q_sel = posts.loc[:, 'PostTypeId'] == 1
questions = posts.loc[q_sel, :]

In [7]:
# Isolate answers in a separate data frame.
a_sel = posts.loc[:, 'PostTypeId'] == 2
answers = posts.loc[a_sel, :]

In [8]:
# Dates of first answers.
answer_dates = answers.groupby('ParentId')['CreationDate'].min().rename('AnswerDate')

# Merge first answer dates with questions.
questions = questions.merge(
    answer_dates,
    how='left',
    left_on='Id',
    right_index=True
)

In [9]:
# Question answer time.
answer_time = questions['AnswerDate'] - questions['CreationDate']

# Target column: was the question answered within 7 days?
questions['Answered7d'] = answer_time.dt.days <= 7

---

We wish to engineer several features for our predictive model. We first extract the hour of the day during which a question was posted, since questions posted during odd hours are more liable to be buried.

In [10]:
# Get hour of day from question creation time.
questions['CreationTime'] = questions['CreationDate'].dt.hour

We include the word count of question titles as a feature. Extremely short titles are unlikely to be adequately descriptive, while overly long titles may be cumbersome to read.

In [11]:
# Count number of words in question title.
questions['TitleWords'] = questions['Title'].str.split().apply(len)

Adding tags to a question makes the question easier to search and can attract answerers interested in certain topics. We therefore include the number of tags as a feature.

In [12]:
# Number of tags in questions.
questions['NumTags'] = questions['Tags'].str.findall('<.*?>').apply(len)

Overall activity on the Stack Exchange is likely to affect the probability of receiving a timely answer. If fewer questions are being asked, there is less competition and thus one's odds of receiving an answer may be higher. Likewise, if more answers are being posted, any particular question is more likely to receive an answer. We separately count the number of questions and answers on each day, take the 30-day rolling averages, and use the averages from the day prior to question posting as features.

In [13]:
# Create time series indexed by question creation date.
questions_time = questions.set_index(
    'CreationDate',
    drop=True
).sort_index()

# Get rolling 30-day average of daily question numbers.
daily_questions = questions_time.loc[:, 'PostTypeId'] \
    .resample('D').count().rename('AvgDailyQuestionCount') \
    .rolling(window='30D').mean()

In [14]:
# Create time series indexed by answer creation date.
answers_time = answers.set_index(
    'CreationDate',
    drop=True
).sort_index()

# Get rolling 30-day average of daily answer numbers.
daily_answers = answers_time.loc[:, 'PostTypeId'] \
    .resample('D').count().rename('AvgDailyAnswerCount') \
    .rolling(window='30D').mean()

In [15]:
# Get shifted 30-day average question count.
avg_question_count= pd.Series(
    index=questions['CreationDate'].dt.date,
    data=daily_questions.shift(1).fillna(0)
).reset_index(drop=True)
avg_question_count.index = questions.index
questions['AvgDailyQuestionCount'] = avg_question_count

# Get shifted 30-day average answer count.
avg_answer_count = pd.Series(
    index=questions['CreationDate'].dt.date,
    data=daily_answers.shift(1).fillna(0)
).reset_index(drop=True).fillna(0)
avg_answer_count.index = questions.index

# Create columns for prior day's 30-day average question/answer count.
questions['AvgDailyQuestionCount'] = avg_question_count
questions['AvgDailyAnswerCount'] = avg_answer_count

It is not uncommon for people to leave comments on a question instead of an answer, especially if the question needs further clarification. Responding to such comments can often lead to an answer from the commenter. We therefore count the number of comments left by the question poster which were made before the first answer (or within 7 days of the question posting if the question was not answered within 7 days).

In [16]:
def find_comments(row):
    '''
    Find the number of comments on a question left by the question poster
    within 7 days (if the question is unanswered) or prior to the first answer.

    Arguments
    ---------
    row - pandas.Series
        A row of the a DataFrame containing Stack Exchange questions.

    Returns
    -------
    num_comments - int
        The number of comments counted.
    '''
    # Get id of question poster.
    id = row['OwnerUserId']

    # Get cutoff date for comments.
    date = row['AnswerDate']
    date = row['CreationDate'] + pd.DateOffset(7) if pd.isnull(date) else date

    # Count number of comments before cutoff date.
    sel = (comments['UserId'] == id) & (comments['CreationDate'] < date)
    num_comments = len(comments[sel])

    return num_comments

In [ ]:
# Count numner of comments.
questions['NumComments'] = questions.apply(
    lambda row: find_comments(row),
    axis=1
).fillna(0)

A question asker's prior history on the Stack Exchange may be relevant to their odds of getting a question answered. Users may be uninclined to answer a question asked by someone with a reputation for asking poor or low effort questions. Likewise, users who have contribtued answers of their own may be more likely to get answers to their own questions. We engineer features to separately count the number of prior questions and answers asked by the poster, as well as the average scores of said questions and answers.

In [18]:
def find_prior_questions(row):
    '''
    Given a DataFrame row corresponding to a particular Stack Exchange question
    , get all questions and their scores given by the question poster prior to the
    particular question.

    Arguments
    ---------
    row - pandas.Series
        A row of the a DataFrame containing Stack Exchange questions.

    Returns
    -------
    prior_questions - pandas.DataFrame
        A DataFrame containing all prior questions from the question poster and
        their scores.
    '''
    # Get id of question poster.
    id = row['OwnerUserId']

    # Find questions from the same user with date prior to the question date.
    date = row['CreationDate']
    sel = (questions['OwnerUserId'] == id) & (questions['CreationDate'] < date)
    prior_questions = questions.loc[sel, 'Score']
    
    return prior_questions

In [17]:
def find_prior_answers(row):
    '''
    Given a DataFrame row corresponding to a particular Stack Exchange question
    , get all answers and their scores given by the question poster prior to the
    particular question.

    Arguments
    ---------
    row - pandas.Series
        A row of the a DataFrame containing Stack Exchange questions.

    Returns
    -------
    prior_answers - pandas.DataFrame
        A DataFrame containing all prior answers from the question poster and
        their scores.
    '''
    # Get id of question poster.
    id = row['OwnerUserId']

    # Find answers from the same user with date prior to the question date.
    date = row['CreationDate']
    sel = (answers['OwnerUserId'] == id) & (answers['CreationDate'] < date)
    prior_answers = answers.loc[sel, 'Score']
    
    return prior_answers

In [19]:
# Number of prior questions.
questions['NumPriorQuestions'] = questions.apply(
    lambda row: len(find_prior_questions(row)),
    axis=1
).fillna(0)

# Average score of prior questions.
questions['AvgQuestionScore'] = questions.apply(
    lambda row: find_prior_questions(row).mean(),
    axis=1
).fillna(0)

# Number of prior answers.
questions['NumPriorAnswers'] = questions.apply(
    lambda row: len(find_prior_answers(row)),
    axis=1
).fillna(0)

# Average score of prior answers.
questions['AvgAnswerScore'] = questions.apply(
    lambda row: find_prior_answers(row).mean(),
    axis=1
).fillna(0)

We engineer a number of features which measure the types of content found in a question. These include the number of lines of code, the number of displayed math equations, and the number of lines of text in the post. We also count the number of various HTML tags that appear in questions, including `blockquote`, `ul`, and `img`.

The functions used to extract these features are in the imported Python module `se_post_processing.py`, which can be found in the `src` folder of the project directory.

In [20]:
# Count number of lines of code.
questions['CodeLines'] = questions['Body'] \
    .apply(sepp.count_code_lines)

# Count number of lines of text (excluding math and code).
questions['TextLines'] = questions['Body'] \
    .apply(sepp.count_text_lines)

# Count number of lines of displayed math equations.
questions['MathEquations'] = questions['Body'] \
    .apply(lambda x: len(sepp.find_math(x)))

# Count number of quote HTML tags.
questions[ 'Quotes'] = questions['Body'] \
    .apply(lambda x: sepp.count_html_tags(x, 'blockquote'))

# Count number of bullet list HTML tags.
questions['BulletLists'] = questions['Body'] \
    .apply(lambda x: sepp.count_html_tags(x, 'ul'))

# Count number of image HTML tags.
questions['Images'] = questions['Body'] \
    .apply(lambda x: sepp.count_html_tags(x, 'img'))

---

# 3. Modeling
---

With our data processing complete, we move on to modeling. The first step is to separate our data into training and test sets. We extract a random sample of $20\%$ of the data for testing.

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
# Columns to drop.
nonfeature_cols = [
    'Answered7d',
    'Tags',
    'Id',
    'PostTypeId',
    'CreationDate',
    'Score',
    'ViewCount',
    'Body',
    'OwnerUserId',
    'LastActivityDate',
    'Title',
    'AnswerCount',
    'CommentCount',
    'ClosedDate',
    'ContentLicense',
    'AcceptedAnswerId',
    'LastEditorUserId',
    'LastEditDate',
    'ParentId',
    'OwnerDisplayName',
    'CommunityOwnedDate',
    'LastEditorDisplayName',
    'FavoriteCount',
    'AnswerDate'
]

# Separate features and target variable.
X = questions.drop(nonfeature_cols, axis=1)
y = questions['Answered7d']

# Split into training and test sets.
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2)

---

We are ready to train our model. We use a random forest model to predict whether or not a question is answered within 7 days. We found that random forest performed better than logistic regression, while still being reasonably interpretable. The second point is important, as we would like to draw inferences from our model about how Stack Exchange users can structure their questions to maximize the odds of getting a timely answer. We use a cross-validated grid search to optimize the hyperparameters of our random forest.

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [28]:
# Create pipeline
steps = [('model', RandomForestClassifier())]
pipeline = Pipeline(steps)

# Parameter grid
param_grid = [{
    'model': [RandomForestClassifier(n_estimators=1000)],
    'model__max_depth': range(5, 55, 5),
    'model__max_features': range(1, 17),
    'model__min_samples_split': range(2, 12, 2)
}]

# Perform grid search.
grid = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('model', RandomForestClassifier())]),
             n_jobs=-1,
             param_grid=[{'model': [RandomForestClassifier(max_depth=10,
                                                           max_features=9,
                                                           min_samples_split=6,
                                                           n_estimators=1000)],
                          'model__max_depth': range(5, 55, 5),
                          'model__max_features': range(1, 17),
                          'model__min_samples_split': range(2, 12, 2)}])

In [34]:
# Model predictions.
train_predict = grid.predict(X_train)
test_predict = grid.predict(X_test)

# Training classification report.
train_cr = classification_report(y_train, train_predict)

# Test classification report.
test_cr = classification_report(y_test, test_predict)

# Print classification reports.
print('Train:')
print(train_cr)
print()
print('Test:')
print(test_cr)

Train:
              precision    recall  f1-score   support

       False       0.73      0.36      0.48     10075
        True       0.74      0.93      0.82     19358

    accuracy                           0.73     29433
   macro avg       0.73      0.64      0.65     29433
weighted avg       0.73      0.73      0.71     29433


Test:
              precision    recall  f1-score   support

       False       0.62      0.30      0.41      2549
        True       0.71      0.90      0.79      4810

    accuracy                           0.69      7359
   macro avg       0.66      0.60      0.60      7359
weighted avg       0.68      0.69      0.66      7359



The model performed reasonably well in terms of F1-score on the `1` class (question answered within 7 days). However, recall on the `0` class was poor in the test set. However, we note an improvement over our baseline model in terms of both recall on the `0` class and accuracy. In particular, the test accuracy is now higher than the majority class share.

In [ ]:
# Feature importances
feat_imp = pd.DataFrame(
    index=X_train.columns,
    columns=['Importance'],
    data=rfc.feature_importances_
).sort_values('Importance', ascending=False)
feat_imp

# 4. Conclusion
---

Although our baseline can reasonably predict the positive outcome (a question is answered within 7 days), it is much less effective at predicting the negative outcome. However, our model's predictions are still more accurate than those of the constant model (guessing that every question is answered within 7 days).